In [3]:
!pip install transformers
!pip install torch
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00


## **Installing Modules**

In [4]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

## **Model Initialization**

In [5]:
# Initializing T5 model - used t5 small varient
model = T5ForConditionalGeneration.from_pretrained('t5-small') ##-----T5 Model for text related tasks
tokenizer = T5Tokenizer.from_pretrained('t5-small',legacy=True) ##-------For tokenizing text, (specifically designed for T5)
device = torch.device('cpu')

## **Input Text**

In [6]:
# Input Text
text = """
On 22 July 2019, ISRO launched Chandrayaan-2 on board a Launch Vehicle Mark-3 (LVM3) launch vehicle consisting of an orbiter, a lander and a rover.[15] The lander was scheduled to touch down on the lunar surface on 6 September 2019 to deploy the Pragyan rover. The lander ultimately crashed when it lost contact with earth (ISRO) and deviated from its intended trajectory while attempting to land near the lunar south pole.[16][17]

The lunar South Pole region holds particular interest for scientific exploration due to studies that show large amounts of ice there. Mountainous terrain and unpredictable lighting conditions not only protect the ice from melting, but also make landing scientific probes there a challenging undertaking. This ice could contain solid-state compounds that would normally melt under warmer conditions elsewhere on the Moon, compounds which could provide insight into lunar, Earth, and Solar System history. Ice could also be used as a source of drinking water and hydrogen for fuel and oxygen for future manned missions and outposts.[18][19]

The European Space Tracking network (ESTRACK), operated by the European Space Agency (ESA), is supporting the mission. Under a new cross-support arrangement, ESA tracking support could be provided for upcoming ISRO missions such as those of India's first human spaceflight programme, Gaganyaan, and the Aditya-L1 solar research mission. In return, future ESA missions will receive similar support from ISRO's own tracking stations.
"""

## **Pre-Processing**

In [7]:
# Preprocessed the text for T5 model input
preprocessed_text = text.strip().replace('\n','') ##---remove leading & trailing space, replace "\n" by ""
t5_input_text = 'summarize: '+ preprocessed_text ##----- adding prefix

In [8]:
# Length of text input
len(t5_input_text.split())

228

In [9]:
# Tokenization of input text
tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device) ##---returns Pytorch tensors

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


## **Generate Summary**

In [10]:
# Generating summary within 30-120 word limit
summary_ids = model.generate(tokenized_text, min_length=30, max_length=120)
# Decoding the generated summary back into human readable
summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True) #----skipping special tokens

In [11]:
# output summary
summary

'lander was scheduled to touch down on the lunar surface on 6 September 2019. it crashed when it lost contact with earth and deviated from its intended trajectory. ice could be used as a source of drinking water and hydrogen for fuel and oxygen.'

## **Streamlit Application**

In [12]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 21.8 MB/s eta 0:00:00


In [13]:
# Creating app file
%%writefile app.py
import streamlit as st
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

st.title('Text Summarization')

# Initializing T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

user_input = st.text_area("Enter your text here", "")

if st.button("Generate Summary"):
  if user_input:
    # Pre-processing
    preprocessed_text = user_input.strip().replace('\n','')
    t5_input_text = 'summarize: '+ preprocessed_text

    # Tokenization
    tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)

    # Generating summary
    summary_ids = model.generate(tokenized_text, min_length=30, max_length=120)
    summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True)
    st.write("Summary: ")
    st.write(summary)
  else:
    st.warning("Please enter some text.")

Writing app.py


In [14]:
!wget -q -O - ipv4.icanhazip.com

34.125.49.65


In [15]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] | fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.49.65:8501

npx: installed 22 in 8.732s
your url is: https://polite-actors-attend.loca.lt
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strate